In [1]:
import os
from pydantic_ai.settings import ModelSettings

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen3-30B-A3B", settings=ModelSettings(temperature=0.0),provider=provider)

Config set: http://localhost:8000/v1


In [2]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model
)
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output
await run_async("What is the capital of France?")

'\n\nThe capital of France is **Paris**. It is a major city in Europe, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. Paris serves as the political, economic, and cultural center of the country.'

In [3]:
from typing import List, Dict
from pydantic_ai import Tool
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import dateutil.parser

@Tool
def retrieve_calendar_events(user: str, start: str, end: str) -> List[Dict]:
    """
    Retrieve calendar events for a specific user between start and end times.
    
    Args:
        user: Email address of the user whose calendar to check
        start: Start time in ISO format (e.g., '2023-12-01T09:00:00')
        end: End time in ISO format (e.g., '2023-12-01T17:00:00')
        
    Returns:
        List of event dictionaries with StartTime, EndTime, NumAttendees, Attendees, and Summary
    """
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(
        calendarId='primary', 
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    for event in events:
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except KeyError: 
            attendee_list.append("SELF")
        start_time = event["start"].get("dateTime", event["start"].get("date"))
        end_time = event["end"].get("dateTime", event["end"].get("date"))
        events_list.append({
            "StartTime": start_time, 
            "EndTime": end_time, 
            "NumAttendees": len(set(attendee_list)), 
            "Attendees": list(set(attendee_list)),
            "Summary": event.get("summary", "No title")
        })
    return events_list

In [4]:
agent = Agent(
    model=agent_model,
    tools=[retrieve_calendar_events],
    system_prompt="""
    You are an AI scheduling assistant. Follow these steps:

    1. **Input Analysis**:
       - Parse the meeting request:
         - Organizer: {From}
         - Attendees: {Attendees[*].email}
         - Duration: 1 hour (from EmailContent)
         - Preferred day: Monday (from EmailContent)
         - Location: {Location}

    2. **Priority Meetings** (marked urgent/important):
    - check EmailContent for keywords: ["urgent", "priority", "ASAP", "important"]
       - If found, escalate to priority Meeting
       - NEVER reschedule the priority meeting request
       - MUST reschedule conflicting NON-PRIORITY events
       - Preserve all existing PRIORITY events
       - Add new priority meeting at requested time

    3. **Regular Meetings**:
       - NEVER reschedule conflicting events
       - Find next available slot

       
    4. **Time Suggestion Logic**:
       - Fetch calendar events for all attendees for:
         - Date range: Preferred day: Monday/Tuesday (from EmailContent) 8AM-6PM IST. The slot will always occur in the future from DateTime (from EmailContent).
         - Timezone: +05:30 (IST)
        - Find which attendee is busy in the preferred slot. The slot will always occur in the future from DateTime (from EmailContent). 
       - Find the earliest Duration in the future: (from EmailContent) slot where:
         1. All attendees are free
         2. Preferably between 10AM-6PM work hours
         3. Avoid lunch hours (1PM-2PM)
         4. Avoid time after 6PM. (decline politely). If no availability on the same day for any time slot beyond 6PM, shift to the next day. 
         5. If the meeting is a priority, then dont reschedule it. Reschedule the conflicting events for the attendee instead.
        
        



    5. **Output Format** (JSON):
       ```json
        updated list for all attendeees with the meeting details. if the meeting is priority, then reschedule the existing event
        for the busy attendee, and print their new schedule. 
            "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5 (from email content)",
            "Datetime": "DateTime (from EmailContent)",
            "Location": "Location (from EmailContent)",
            "From": "userone.amd@gmail.com (from EmailContent)",
            "Attendees": [
                {  you have to update the organizers calendar and the attendees calendar and print in this format. It is mandatory to print all 3 attendees updated schedule.
                    "email": "userone.amd@gmail.com",
                    "events": [
                        {
                        if any existing evet is rescheduled, to schedule a priority event, then print both the updated events in this format
                             "StartTime": "YYYY-MM-DDTHH:MM:SS+05:30",
                               "EndTime": "YYYY-MM-DDTHH:MM:SS+05:30" (of the proposed meeting slot)
                            "NumAttendees": 3,
                            "Attendees": [
                                "userone.amd@gmail.com", (from email content)
                                "usertwo.amd@gmail.com",
                                "userthree.amd@gmail.com"
                            ],
                            "Summary": "Agentic AI Project Status Update"
                        }
                    ]

                    "email": "usertwo.amd@gmail.com",
                    "events": [
                        {
                            if any existing event is rescheduled, to schedule a priority event, then print both the updated events in this format
                             "StartTime": "YYYY-MM-DDTHH:MM:SS+05:30",
                               "EndTime": "YYYY-MM-DDTHH:MM:SS+05:30" (of the proposed meeting slot)
                            "NumAttendees": 3,
                            "Attendees": [
                                "userone.amd@gmail.com", (from email content)
                                "usertwo.amd@gmail.com",
                                "userthree.amd@gmail.com"
                            ],
                            "Summary": "Agentic AI Project Status Update"
                        }
                        
                    ]
                    
                    "email": "userthree.amd@gmail.com",
                    "events": [
                        {
                        if any existing event is rescheduled, to schedule a priority event, then print both the updated events in this format
                             "StartTime": "YYYY-MM-DDTHH:MM:SS+05:30",
                               "EndTime": "YYYY-MM-DDTHH:MM:SS+05:30" (of the proposed meeting slot)
                            "NumAttendees": 3,
                            "Attendees": [
                                "userone.amd@gmail.com", (from email content)
                                "usertwo.amd@gmail.com",
                                "userthree.amd@gmail.com"
                            ],
                            "Summary": "Agentic AI Project Status Update"
                        }
                    ]
                    
                },
                
        "MetaData": {
        "Subject": "Agentic AI Project Status Update",
        "Location": "Virtual",
        "Organizer": "organizer@example.com",
        "Attendees": ["userone@example.com", "usertwo@example.com", "userthree@example.com"]
         },
        "PoliteEmailDraft":{
            "Subject": "Proposed Meeting: Agentic AI Project",
            "Body" : I have scheduled the meeting at scheduled time and date (of the proposed meeting slot suggested by you ) .. Please attend.
       }
       ```
    """
)

In [5]:
res = await run_async("""
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project." 
    let everyone know the meeting is scheduled. 
""")
import json
res = json.loads(res)
(res)

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '19-07-2025T12:34:55',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'userone.amd@gmail.com',
   'events': [{'StartTime': '2025-07-24T10:30:00+05:30',
     'EndTime': '2025-07-24T11:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Agentic AI Project Status Update'}]},
  {'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-24T10:30:00+05:30',
     'EndTime': '2025-07-24T11:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Agentic AI Project Status Update'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-24T10:30:00+05:30',
     'EndTime': '2025-07-24T11:00:00+05:30',
     'NumAttendees': 3,
     'Attend

In [6]:
res = await run_async("""
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Validation - Urgent",
    "EmailContent": "Hi Team. We’ve just received quick feedback from the client indicating that the instructions we provided aren’t working on their end. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM to discuss and resolve this issue."
""")
import json
res = json.loads(res)
(res)

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033b5',
 'Datetime': '19-07-2025T12:34:55',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'userone.amd@gmail.com',
   'events': [{'StartTime': '2025-07-21T09:00:00+05:30',
     'EndTime': '2025-07-21T10:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Client Validation - Urgent'}]},
  {'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-21T09:00:00+05:30',
     'EndTime': '2025-07-21T10:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Client Validation - Urgent'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-21T09:00:00+05:30',
     'EndTime': '2025-07-21T10:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['user

In [7]:
res = await run_async("""
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
""")
import json
res = json.loads(res)
(res)

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033c5',
 'Datetime': '2025-07-22T11:00:00+05:30',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'userone.amd@gmail.com',
   'events': [{'StartTime': '2025-07-22T11:00:00+05:30',
     'EndTime': '2025-07-22T12:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Project Status'}]},
  {'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-22T11:00:00+05:30',
     'EndTime': '2025-07-22T12:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userthree.amd@gmail.com'],
     'Summary': 'Project Status'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-22T11:00:00+05:30',
     'EndTime': '2025-07-22T12:00:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userone.amd@gmail.com'